In [ ]:
from pathlib import Path
import subprocess
from typing import Union, Optional


def convert_mkv_to_mp4(
    input_mkv: Union[str, Path],
    output_mp4: Optional[Union[str, Path]] = None,
    overwrite: bool = True,
    quiet: bool = False,
    copy_streams: bool = True,
    convert_subtitles: bool = True,
    faststart: bool = True,
) -> bool:
    """
    Convert MKV file to MP4 using ffmpeg (preferably via stream copy / remux).

    Parameters
    ----------
    input_mkv : str | Path
        Path to input .mkv file

    output_mp4 : str | Path, optional
        Output path. If None → same name + .mp4

    overwrite : bool
        Overwrite existing output file

    quiet : bool
        Suppress most ffmpeg output

    copy_streams : bool
        True  → fast remux (copy video/audio streams) – recommended
        False → re-encode to H.264/AAC (slower, fixes compatibility issues)

    convert_subtitles : bool
        When copy_streams=True: try to convert common subtitle formats
        (ASS, SRT, etc.) to mov_text so they work in MP4

    faststart : bool
        Move metadata to beginning of file → better for streaming/online playback

    Returns
    -------
    bool : True if conversion succeeded
    """
    input_path = Path(input_mkv).resolve()
    if not input_path.is_file():
        print(f"Error: Input file not found → {input_path}")
        return False

    if output_mp4 is None:
        output_path = input_path.with_suffix(".mp4")
    else:
        output_path = Path(output_mp4).resolve()

    output_path.parent.mkdir(parents=True, exist_ok=True)

    if output_path.exists() and not overwrite:
        print(f"Skipping (already exists): {output_path.name}")
        return True

    print(f"Converting: {input_path.name}")
    print(f"       →    {output_path.name}")

    cmd = ["ffmpeg", "-i", str(input_path)]

    if copy_streams:
        # Fast remux – no quality loss
        cmd.extend(["-map", "0", "-c:v", "copy", "-c:a", "copy"])

        # Most common subtitle fix for MP4
        if convert_subtitles:
            cmd.extend(["-c:s", "mov_text"])
        else:
            cmd.extend(["-sn"])  # drop subtitles completely

        # Many players / streaming like this flag
        if faststart:
            cmd.extend(["-movflags", "+faststart"])
    else:
        # Re-encode fallback (slower but more compatible)
        cmd.extend([
            "-map", "0",
            "-c:v", "libx264", "-preset", "medium", "-crf", "23",
            "-c:a", "aac", "-b:a", "192k",
        ])
        if convert_subtitles:
            cmd.extend(["-c:s", "mov_text"])

    if overwrite:
        cmd.append("-y")

    if quiet:
        cmd.extend(["-loglevel", "error", "-nostats"])

    cmd.append(str(output_path))

    try:
        subprocess.run(
            cmd,
            check=True,
            capture_output=quiet,
            text=True,
        )
        print("Done.")
        return True

    except subprocess.CalledProcessError as e:
        print(f"Failed: {input_path.name}")
        if not quiet and e.stderr:
            print("ffmpeg error output:")
            print(e.stderr.strip())
        return False

    except FileNotFoundError:
        print("Error: ffmpeg not found.")
        print("Install ffmpeg → https://ffmpeg.org/download.html")
        print("   • Linux:   sudo apt install ffmpeg")
        print("   • macOS:   brew install ffmpeg")
        print("   • Windows: chocolatey install ffmpeg  or download binary")
        # winget install -e --id Gyan.FFmpeg
        # ffmpeg -version
        return False


In [2]:
# ────────────────────────────────────────────────
# Example calls (you can remove or comment this part)
# ────────────────────────────────────────────────

if __name__ == "__main__":
    # # Single file – fastest method
    # convert_mkv_to_mp4("RSaab.mkv")

    # Different output name + quiet mode
    convert_mkv_to_mp4(
        "RSaab.mkv",
        "RSaab_converted.mp4",
        quiet=True
    )

    # # Batch convert folder
    # folder = Path("videos")
    # for mkv_file in folder.glob("**/*.mkv"):
    #     convert_mkv_to_mp4(mkv_file, quiet=True)

Converting: RSaab.mkv
       →    RSaab_converted.mp4
Done.
